In [37]:
import aoc.common.process_input
import aoc.year2022.day07
fileContent = aoc.common.process_input.read_file("input\\year2022\\day07\\example.txt")        
input_content = aoc.common.process_input.to_function_list(fileContent, str)

In [38]:
# tree
tree = {}
root = '/'
# each node
# dir: <name> = (d, parent_dir, <size>, <list of nodes inside>)
# file: <name> = (f, parent_dir, <size>)
current_dir = root
tree[root] = ('d', None, 0, [])
i_type = 0
i_parent = 1
i_size = 2
i_list = 3

In [39]:
for input in input_content:
    to_print = input
    parts = input.split(' ')
    if '$' in input: # it is a command
        if parts[1] == 'cd':
            if parts[2] == "..":
                to_print += ' # move up a dir'
                # move up a dir
                if current_dir == root:
                    raise Exception("Can't move up from root!")
                current_dir = tree[current_dir][i_parent]
            else:
                to_print += ' # move into a dir'
                # move into a dir
                if parts[2] == root:
                    # don't need to do anything, this is the initial state
                    continue
                else:
                    # can only move to a dir that is below the current_dir
                    new_dir = current_dir + "/" + parts[2]
                    if new_dir not in tree[current_dir][i_list]:
                        raise Exception("Can't move to dir " + new_dir + " from " + current_dir)
                    else:
                        current_dir = new_dir
        elif parts[1] == 'ls':
            to_print += ' # list dir'
            # current_dir should be a dir otherwise it will bork
            if tree[current_dir][i_type] != 'd':
                raise Exception('Invalid ls: ' + current_dir + ' is not a dir.')
        else:
            raise Exception('Invalid Command: ' + parts[1])
    else:
        name = parts[1]
        fullpath = current_dir + "/" + name
        if parts[0] == 'dir':
            to_print += ' # it is a dir'
            size = 0
            # new node in the tree
            tree[fullpath] = ('d', current_dir, size, [])
        else: 
            to_print += ' # it is a file'
            size = int(parts[0])
            # new node in the tree
            tree[fullpath] = ('f', current_dir, size)
        # need to add it to the current_dir
        current = tree[current_dir]
        file_list = current[i_list]
        file_list.append(fullpath)
        tree[current_dir] = (current[i_type], current[i_parent], current[i_size], file_list)
    # print(to_print)
# print(tree)


Exception: Can't move to dir //a from /

In [ ]:
for key in tree:
    print(f"{key}: {tree[key]}")


/: ('d', None, 0, ['//a', '//b.txt', '//c.dat', '//d'])
//a: ('d', '/', 0, ['//a/e', '//a/f', '//a/g', '//a/h.lst'])
//b.txt: ('f', '/', 14848514)
//c.dat: ('f', '/', 8504156)
//d: ('d', '/', 0, ['//d/j', '//d/d.log', '//d/d.ext', '//d/k'])
//a/e: ('d', '//a', 0, ['//a/e/i'])
//a/f: ('f', '//a', 29116)
//a/g: ('f', '//a', 2557)
//a/h.lst: ('f', '//a', 62596)
//a/e/i: ('f', '//a/e', 584)
//d/j: ('f', '//d', 4060174)
//d/d.log: ('f', '//d', 8033020)
//d/d.ext: ('f', '//d', 5626152)
//d/k: ('f', '//d', 7214296)


In [ ]:
# calculate dir size
def calculate_dir_size(tree, dir):
    dir_node = tree[dir]
    if dir_node[i_type] == 'f':
        return dir_node[i_size]
    if dir_node[i_size] != 0:
        return dir_node[i_size]
    size = 0
    for sub_node in dir_node[i_list]:
        node = tree[sub_node]
        if node[i_size] != 0:
            size += node[i_size]
        else:
            size += calculate_dir_size(tree, sub_node)
    # update size
    tree[dir] = (dir_node[i_type], dir_node[i_parent], size, dir_node[i_list])
    return size

calculate_dir_size(tree, root)



48381165

In [ ]:
result = 0
for key in tree.keys():
    node = tree[key]
    if node[i_type] == 'd':
        if node[i_size] <= 100000:
            result += node[i_size]
print(result)

95437
